# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20901989


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:01<00:20,  1.39it/s]

Rendering models:  10%|▉         | 3/31 [00:02<00:22,  1.23it/s]

Rendering models:  16%|█▌        | 5/31 [00:03<00:20,  1.27it/s]

Rendering models:  32%|███▏      | 10/31 [00:04<00:11,  1.79it/s]

...

Rendering models:  68%|██████▊   | 21/31 [00:07<00:02,  3.34it/s]

Rendering models:  77%|███████▋  | 24/31 [00:07<00:01,  4.53it/s]

Rendering models:  87%|████████▋ | 27/31 [00:07<00:00,  6.03it/s]

Rendering models:  97%|█████████▋| 30/31 [00:07<00:00,  7.62it/s]

equidad1                              0.001996
Bertrand15                            0.008786
not-logged-in-e0d22fca112ab22e0f7f    0.002949
CTidwell3                             0.000311
not-logged-in-204fc099209d1816fe0b    0.000348
bkbolli                               0.001159
kenyandall                            0.002875
not-logged-in-076d4b51a9c3210cfd57    0.002455
not-logged-in-75af55cf19be29c510aa    0.146192
Jboy2000                              0.673355
not-logged-in-f09819078c043556a4da    0.000682
browny811                             0.000858
Jessica_Korkmaz                       0.000965
not-logged-in-c7d2e7cd49f9072ed0e6    0.485147
not-logged-in-efc6ca268a17c35083af    0.194586
not-logged-in-d04ed8c60cd5453cb408    0.000518
AlexYoung35                           0.000451
acapirala                             0.000316
pangeli5                              0.000222
Mr_DMan                               0.003539
Lavadude                              0.051993
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())